In [1]:
pip install transformers

     |████████████████████████████████| 2.5MB 19.7MB/s 
     |████████████████████████████████| 3.3MB 24.5MB/s 
     |████████████████████████████████| 901kB 44.1MB/s 


In [33]:
!git clone "Repo URL" --quiet

In [34]:
import pandas as pd
from torch.utils.data import DataLoader , Dataset
from sklearn.model_selection import train_test_split
import os
import random
import math
import torch

In [35]:
cd MTLVS/Codes

/content/MTLVS/Codes


In [36]:
dfc = pd.read_pickle("../data/features/summary_dataframes/dfc_0.57.pkl")
dfg = pd.read_pickle("../data/features/summary_dataframes/dfg_0.72.pkl")
dfo = pd.read_pickle("../data/features/summary_dataframes/dfo_0.6.pkl")
dfs = pd.read_pickle("../data/features/summary_dataframes/dfs_0.6.pkl")

In [37]:
dfc.head()

,Event,Tweet_ID,User_ID,Date,Orig_Tweet,Clean_Tweet,Norm_Tweet,SR,R1NR0,False0_True1_Unveri2_NR3_Rep4,Situ1_NonSitu0_Oth2,Summary_gt,Stance,StanceLabel,New_Summ_gt_Clean,New_Summ_gt_Clean_Situ,New_Summ_gt_BT,New_Summ_gt_BT_Situ
0,charliehebdo,552789127792717826,297325085,2015-01-07 11:29:33+00:00,Just confirmed. 10 people dead at shooting in ...,Just confirmed 10 people dead at shooting in t...,Just confirmed . 10 people dead at shooting in...,S,1,1,1,0,none,2,0,0,0,0
1,charliehebdo,552801880812691456,220416676,2015-01-07 12:20:13+00:00,Cartoonist Stephane Charbonnier was critically...,Cartoonist Stephane Charbonnier was critically...,Cartoonist Stephane Charbonnier was critically...,S,1,0,1,0,none,2,0,0,0,0
2,charliehebdo,553552825431883776,2566535282,2015-01-09 14:04:12+00:00,Breaking: Unconfirmed reports of a gunman at t...,Breaking Unconfirmed reports of a gunman at th...,Breaking : Unconfirmed reports of a gunman at ...,S,1,0,0,0,none,2,0,0,0,0
3,charliehebdo,553589055116836864,3108351,2015-01-09 16:28:10+00:00,Breaking: Kouachi brothers dead; hostage freed...,Breaking Kouachi brothers dead hostage freed a...,Breaking : Kouachi brothers dead ; hostage fre...,S,1,1,1,0,none,2,0,0,0,0
4,charliehebdo,552818821460148226,25049056,2015-01-07 13:27:32+00:00,According to #CharlieHebdo's lawyer four well-...,According to CharlieHebdo s lawyer four well k...,According to #CharlieHebdo 's lawyer four well...,S,1,2,1,0,none,2,0,0,0,0


In [38]:
dfmain = pd.concat([dfc,dfg,dfo,dfs],ignore_index= True)

In [39]:
class Place:
    def __init__(self,df,replicate=5):
        self.df = df
        self.replicate = replicate-1
        self.tweetidlist = random.sample(list(df.Tweet_ID), k=len(df))
    def __len__(self):
        return (self.replicate + 1)*len(self.df)

    def get(self, docsize):
        if not self.tweetidlist and not self.replicate:
            return None
            # raise Exception("Tweets exhausted")
        tweetidlist,self.tweetidlist = self.tweetidlist[:docsize],self.tweetidlist[docsize:]
        if not self.tweetidlist and self.replicate:
            self.tweetidlist = random.sample(list(self.df.Tweet_ID), k=len(self.df))
            self.replicate-=1
        df = self.df[self.df["Tweet_ID"].isin(tweetidlist)]
        df = df.sort_values("Date", ignore_index=True)
        return df

In [40]:
class Custom_Dataset(Dataset):
    def __init__(self, placelist, docsize):
        self.placelist = placelist
        self.docsize = docsize

    def __len__(self):
        s = sum([math.ceil(len(p)/self.docsize) for p in placelist])
        return s
    
    def __getitem__(self, index):
        place = random.choice(self.placelist)
        df = place.get(self.docsize)
        while df is None:
            self.placelist.remove(place)
            place = random.choice(self.placelist)
            df = place.get(self.docsize)
        tweetid = "\n".join(map(str,list(df.Tweet_ID)))
        doc = "\n".join(list(df.Clean_Tweet))
        labels = "\n".join(map(str,list(df.New_Summ_gt_Clean)))
        summaries = "None"
        # dfdoc = pd.DataFrame({'doc':doc,'labels':labels,'summaries':summaries})
        return (tweetid, doc, labels,summaries)

In [41]:
def createplacelist(traindflist, testdf):
    maxlen = max([len(i) for i in traindflist])
    r = math.ceil(maxlen/64)
    trainplacelist = [Place(i,replicate=r) for i in traindflist]
    testplacelist = [Place(testdf, replicate=r)]

    return trainplacelist, testplacelist

In [42]:
len(dfc), len(dfo) , len(dfg) 

(2079, 890, 469)

In [43]:
traindflist = [dfc,dfo,dfg]
testdf = dfs

In [44]:
placelist, testplacelist = createplacelist(traindflist, testdf)

In [45]:
dataset = Custom_Dataset(placelist,64)

In [46]:
len(dataset)

1773

In [47]:
tweetid = []
doc =[]
labels =[]
summaries =[]
for i in dataset:
    tweetid.append(i[0])
    doc.append(i[1])
    labels.append(i[2])
    summaries.append(i[3])

In [48]:
len (tweetid),len(doc), len(labels) , len(summaries)

(1815, 1815, 1815, 1815)

In [49]:
df = pd.DataFrame({'tweetid': tweetid,'doc':doc,'labels':labels,'summaries':summaries})

In [50]:
df.head()

,tweetid,doc,labels,summaries
0,524923148576518144\n524923341359300608\n524925...,Shooting at the war memorial in Ottawa\nUnifor...,0\n0\n0\n1\n0\n0\n0\n0\n0\n0\n0\n0\n0\n0\n0\n0...,None
1,580318717860532224\n580320745974288384\n580322...,Airbus passenger plane crashed in French Alps ...,0\n0\n1\n0\n0\n0\n0\n0\n0\n0\n1\n1\n0\n1\n0\n0...,None
2,524924225463668738\n524924619812511746\n524925...,BREAKING CTV News confirms that shots have bee...,0\n0\n0\n0\n0\n0\n0\n0\n0\n0\n0\n0\n0\n1\n0\n0...,None
3,580317998147325952\n580319458004193280\n580320...,142 PEOPLE ON BOARD GERMANWINGS AIRBUS A320 TH...,0\n0\n0\n0\n0\n0\n1\n0\n0\n0\n1\n0\n0\n1\n0\n0...,None
4,524922078638903296\n524924987774631936\n524925...,Ottawa police are confirming a shooting at the...,1\n1\n0\n0\n0\n1\n0\n0\n0\n0\n0\n0\n1\n0\n0\n1...,None


In [51]:
from sklearn.model_selection import train_test_split

In [52]:
df_train , df_val = train_test_split(df,test_size = 0.1)

In [53]:
dataset = Custom_Dataset(testplacelist,64)

In [54]:
tweetid = []
doc =[]
labels =[]
summaries =[]
for i in dataset:
    tweetid.append(i[0])
    doc.append(i[1])
    labels.append(i[2])
    summaries.append(i[3])

In [55]:
len (tweetid),len(doc), len(labels) , len(summaries)

(660, 660, 660, 660)

In [56]:
df_test = pd.DataFrame({'tweetid': tweetid,'doc':doc,'labels':labels,'summaries':summaries})

In [57]:
df_test.to_json("data/test.json", orient='records')
df_train.to_json("data/train.json", orient='records')
df_val.to_json("data/val.json", orient='records')

In [58]:
# Training
!python stl_summ.py -device 0 -epochs 5 -batch_size 8 -model RNN_RNN -seed 1 -save_dir "./checkpoints/"

2021-06-18 12:20:12,656 [INFO] Loading vocab,train and val dataset.Wait a second,please
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
RNN_RNN(
  (abs_pos_embed): Embedding(300, 100)
  (rel_pos_embed): Em

In [59]:
# Testing
!python stl_summ.py -device 0 -batch_size 1 -test -load_dir ./checkpoints/RNN_RNN_seed_1.pt

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0% 0/660 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly 

In [60]:
cd ../..

/content
